<a href="https://colab.research.google.com/github/khansajeel/InformationRetrieval_2021_T34_Project/blob/main/backend/IR_Backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask-ngrok
!pip install flask_cors

In [2]:
import pickle
import urllib
from torchvision import transforms
import torch
from PIL import Image
from flask import request
from flask import Flask
from flask_cors import CORS
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import json
import string
import numpy as np
from collections import Counter
from tqdm.notebook import tnrange
from flask_ngrok import run_with_ngrok
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shenv\AppData\Roaming\nltk_data...


In [2]:
class tf_idfmatrices():
    def __init__(self, DFpostings, tokens, docs_count):
        self.DFpostings = DFpostings
        self.tokens = tokens
        self.docs_count = docs_count
        self.vocab_count = len(DFpostings)
        self.vocabulary = [x for x in DFpostings]
        self.idf = dict()
        self.counter_lists = []
        self.tf_idf_TermFreq = np.zeros((docs_count, self.vocab_count))

    def generateIDF(self):
        for key in self.DFpostings:
            doc_freq = len(self.DFpostings[key])
            self.idf[key] = np.log10((self.docs_count/doc_freq) + 1)

    def generateCounterLists(self):
        for i in tnrange(self.docs_count):
            self.counter_lists.append(Counter(self.tokens[i]))

    def generateTermFreq(self):
        for i in tnrange(len(self.vocabulary)):
            for j in range(self.docs_count):
                self.tf_idf_TermFreq[j][i] = (
                    self.counter_lists[j][self.vocabulary[i]]/len(self.tokens[j]))*self.idf[self.vocabulary[i]]


class Results():
    def __init__(self, file_obj, title_obj,
                 file_tf_idf_obj, title_tf_idf_obj):
        self.file_obj = file_obj
        self.title_obj = title_obj
        self.file_tf_idf_obj = file_tf_idf_obj
        self.title_tf_idf_obj = title_tf_idf_obj

    def stripSpecialChar(self, text):
        return ''.join(ch for ch in text if ch.isalnum() and not ch.isdigit() and ch not in string.punctuation)

    def preProcess(self, text):
        stemmer = SnowballStemmer("english")
        stopWords = set(stopwords.words('english'))

        # convert all text to lower case
        text = text.lower()
        # tokenizing the text
        text_tokens = word_tokenize(text)

        stemmedWords = list([stemmer.stem(word) for word in text_tokens])
        validTokens = [i for i in stemmedWords if i not in stopWords]

        # stripping special characters
        validTokens = [self.stripSpecialChar(x) for x in validTokens]
        # Choosing only words which has length > 1
        validTokens = [x for x in validTokens if len(x) > 1]
        return set(validTokens)

    def getscore(self, objtype, listofwords, query_eval):
        indextolookfor = []
        for word in listofwords:
            # instead of forming a query vector, we just extracted the indices of the querytokens
            index = objtype.vocabulary.index(word)
            indextolookfor.append(index)
        for docs in range(objtype.docs_count):
            for query in indextolookfor:
                query_eval[docs][0] += (objtype.tf_idf_TermFreq[docs][query])

    def Top5(self, alist):
        return sorted(range(len(alist)), key=lambda i: alist[i], reverse=True)[:5]

    def getResults(self, sentence_query):
        listofwords = self.preProcess(sentence_query)
        query_eval = np.zeros((self.title_tf_idf_obj.docs_count, 1))
        self.getscore(self.title_tf_idf_obj, listofwords, query_eval)
        self.getscore(self.file_tf_idf_obj, listofwords, query_eval)

        ans = self.Top5(query_eval[:, 0])
        mapping = json.load(open('/content/drive/MyDrive/IR_Project/Final_mapping.json',))
        recipes = json.load(open('/content/drive/MyDrive/IR_Project/recipes.json',))
        data = []

        for i in ans:
            fileid = mapping[i][15:-4]
            recipe=recipes[fileid]["title"]
            data.append({"key": fileid, "title": recipe})
        return data


class InvertedIndex():
    def __init__(self):
        self.DFpostings = {}  # Dictionary contains Posting List for each words
        self.termsInFile = []  # List contains all words in a single file

    def stripSpecialChar(self, text):
        return ''.join(ch for ch in text if ch.isalnum() and not ch.isdigit() and ch not in string.punctuation)

    def preProcess(self, text):
        stemmer = SnowballStemmer("english")
        stopWords = set(stopwords.words('english'))

        # convert all text to lower case
        text = text.lower()
        # tokenizing the text
        text_tokens = word_tokenize(text)

        stemmedWords = list([stemmer.stem(word)
                            for word in text_tokens])   # Stemming
        # Removing Stop Words
        validTokens = [i for i in stemmedWords if i not in stopWords]

        # stripping special characters
        validTokens = [self.stripSpecialChar(x) for x in validTokens]
        # Choosing only words which has length > 1
        validTokens = [x for x in validTokens if len(x) > 1]
        return validTokens, set(validTokens)

    def indexFile(self, file, fileId):
        '''
        Creates Index for each File
        '''
        tokens, setTokens = self.preProcess(file)
        self.termsInFile.append(tokens)
        for i in setTokens:
            if i in self.DFpostings:
                self.DFpostings[i].append(fileId)
            else:
                self.DFpostings[i] = [fileId]

    def save(self, file):
        '''
        Save the index to a file locally
        '''
        json.dump(self.DFpostings, open("DFPostings"+file, "w"))
        json.dump(self.termsInFile, open("TermsInfile"+file, "w"))


In [3]:
app = Flask(__name__)
run_with_ngrok(app)
CORS(app)
app.secret_key = '5ZN5zi!45QUsGG'

@app.route('/', methods=['GET'])
def home():
    return "Information retrieval"


@app.route('/', methods=['POST'])
def index():
    input_image = Image.open(urllib.request.urlopen(request.json['url']))
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0)

    model = torch.load('/content/drive/MyDrive/IR_Project/best_classifier.pt', map_location='cpu')
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')
    with torch.no_grad():
        output = model(input_batch)
    with open("/content/drive/MyDrive/IR_Project/food_labels.txt", "r") as f:
        categories = [s.strip() for s in f.readlines()]

    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    top1_prob, top1_catid = torch.topk(probabilities, 1)

    return {"response": categories[top1_catid[0]]}


@app.route('/results', methods=['POST'])
def comparator():
    sentence_query = request.json['query']
   

    result = Results(file_obj, title_obj,
                     file_tf_idf_obj, title_tf_idf_obj)
    return {"data": result.getResults(sentence_query)}

if __name__ == '__main__':
	print("Loading Models ......")
	with open('/content/drive/MyDrive/IR_Project/ProjectFile.obj', 'rb') as file_object:
		file_obj = pickle.load(file_object)

	print("Done(1/4)")
	with open('/content/drive/MyDrive/IR_Project/ProjectTitle.obj', 'rb') as file_object:
		title_obj = pickle.load(file_object)
	print("Done(2/4)")
	with open('/content/drive/MyDrive/IR_Project/TF_IDF_Calculated_File.obj', 'rb') as file_object:
		file_tf_idf_obj = pickle.load(file_object)
	print("Done(3/4)")
	with open('/content/drive/MyDrive/IR_Project/TF_IDF_Calculated_Title.obj', 'rb') as file_object:
		title_tf_idf_obj = pickle.load(file_object)
	print("Done(4/4)")
	print(" Models Loaded")
	app.run()


NameError: name 'Flask' is not defined